In [ ]:
# docker run --publish=7474:7474 --publish=7687:7687 --volume=/neo4j/data:/data neo4j
# bin/cypher-shell -u neo4j -p <password>
# :exit
# docker run --publish=7474:7474 --publish=7687:7687 -v /neo4j/data:/data neo4j
# neo4j stop

In [ ]:
# https://neo4j.com/labs/genai-ecosystem/graphrag-python/
# https://neo4j.com/blog/genai/what-is-graphrag/

In [41]:
from neo4j import GraphDatabase
from neo4j_graphrag.indexes import create_vector_index, drop_index_if_exists

URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "chuatebongtoi")

INDEX_NAME = "vector-index-name"

# Connect to Neo4j database
driver = GraphDatabase.driver(URI, auth=AUTH)
drop_index_if_exists(
    driver,
    INDEX_NAME,
)
# Creating the index
create_vector_index(
    driver,
    INDEX_NAME,
    label="Document",
    embedding_property="vectorProperty",
    dimensions=3,
    similarity_fn="euclidean",
)

search trên cạnh hoặc node

entity_type (EntityType) – Specifies whether to upsert to nodes (‘NODE’) or relationships (‘RELATIONSHIP’). Defaults to ‘NODE’.

In [46]:
from neo4j_graphrag.indexes import upsert_vectors, retrieve_vector_index_info
# Upsert embeddings data for several nodes
upsert_vectors(
    driver,
    ids=['123', '456', '789'],
    embedding_property="vectorProperty",
    embeddings=[
        [0.12, 0.34, 0.56],
        [0.78, 0.90, 0.12],
        [0.34, 0.56, 0.78],
    ],
    neo4j_database="neo4j",
    entity_type='NODE',
)

In [44]:
from neo4j_graphrag.retrievers import VectorRetriever
retriever = VectorRetriever(driver, index_name=INDEX_NAME, neo4j_database="neo4j")

In [56]:
retrieve_vector_index_info(driver, INDEX_NAME, 'NODE', [0.12, 0.34, 0.56]).data

<bound method Record.data of <Record name='vector-index-name' type='VECTOR' entityType='NODE' labelsOrTypes=['Document'] properties=['vectorProperty'] options={'indexProvider': 'vector-2.0', 'indexConfig': {'vector.hnsw.m': 16, 'vector.hnsw.ef_construction': 100, 'vector.dimensions': 3, 'vector.similarity_function': 'EUCLIDEAN', 'vector.quantization.enabled': True}}>>

In [ ]:
DEFAULT_TEMPLATE = """
You are a top-tier algorithm designed for extracting
information in structured formats to build a knowledge graph.

Extract the entities (nodes) and specify their type from the following text.
Also extract the relationships between these nodes.

Return result as JSON using the following format:
{{"nodes": [ {{"id": "0", "label": "Person", "properties": {{"name": "John"}} }}],
"relationships": [{{"type": "KNOWS", "start_node_id": "0", "end_node_id": "1", "properties": {{"since": "2024-08-01"}} }}] }}

Use only the following nodes and relationships (if provided):
{schema}

Assign a unique ID (string) to each node, and reuse it to define relationships.
Do respect the source and target node types for relationship and
the relationship direction.

Make sure you adhere to the following rules to produce valid JSON objects:
- Do not return any additional information other than the JSON in it.
- Omit any backticks around the JSON - simply output the JSON on its own.
- The JSON object must not wrapped into a list - it is its own JSON object.
- Property names must be enclosed in double quotes

Examples:
{examples}

Input text:

{text}
"""